In [56]:
#구 비율 데이터 만들기 -> PIH_MERGE만들기

In [57]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [58]:
code = pd.read_csv('../Data/processing_data/2nd/busan_code.csv')

In [59]:
sig_code = code[code['DONG'].isna()].reset_index(drop=True)

In [60]:
sig_code = sig_code.drop(columns='DONG')

In [61]:
#불투수면 작업

In [62]:
#전국 데이터에서 부산 데이터를 뽑아냄 불투수면의 경우 법정동코드로 되어 있고 
# 부산의 법정동코드는 21로 시작하는데 부산의 '구' 데이터만 가져온다.
#busan_code = busan_code.rename(columns={'법정동코드':'CODE','n2':'ZONE','n3':'DONG'})

In [63]:
imper = pd.read_excel('../data/rawdata/imper_ratio_data.xlsx')
imper['코드'] = imper['코드'].apply(str)
imper = imper[imper['코드'].str.startswith('21')].reset_index(drop=True)
imper = imper.drop(index=0).reset_index(drop=True)
imper = imper.drop(columns=['코드','기준연도','불투수면 면적(㎢)'])
imper.rename(columns={'시도시군구':'ZONE', '불투수면 비율(%)':'IMP_SUR_RATIO',\
                              '행정구역면적(㎢)':'ZONE_AREA'}, inplace=True)
imper['ZONE_AREA'] = round(imper['ZONE_AREA'],2) * 10000000
imper['IMP_SUR_RATIO'] = round(imper['IMP_SUR_RATIO'],2)
PIH_merge = pd.merge(sig_code, imper, on='ZONE', how='left')

In [64]:
#맨홀작업

In [65]:
manhole = pd.read_excel("../Data/rawdata/manhole_data.xlsx", sheet_name="2-1_지역별 현황", header = 1)

In [66]:
manhole = manhole[manhole["행정구역"] == '부산광역시'][['Unnamed: 2', '합류식맨홀\n(개소)']].iloc[2:]

In [67]:
manhole.rename(columns={'Unnamed: 2':'ZONE', '합류식맨홀\n(개소)':'MANHOLES_COUNT'}, inplace=True)
manhole.reset_index(drop=True)
manhole['MANHOLES_COUNT'] = manhole['MANHOLES_COUNT'].astype(int)

In [68]:
PIH_merge = pd.merge(PIH_merge, manhole, on='ZONE', how='left')

In [69]:
PIH_merge

,CODE,ZONE,IMP_SUR_RATIO,ZONE_AREA,MANHOLES_COUNT
0,2611000000,중구,81.61,3.010000e+07,773
1,2614000000,서구,40.57,1.414000e+08,1308
2,2617000000,동구,57.92,1.009000e+08,2410
3,2620000000,영도구,45.77,1.463000e+08,1914
4,2623000000,부산진구,45.91,2.978000e+08,6581
5,2626000000,동래구,59.03,1.667000e+08,3165
6,2629000000,남구,51.82,2.728000e+08,3156
7,2632000000,북구,22.17,3.976000e+08,6842
8,2635000000,해운대구,27.63,5.232000e+08,8794
9,2638000000,사하구,38.20,4.711000e+08,8331


In [70]:
#펌프작업

In [71]:
pump_data1 = pd.read_csv('../data/rawdata/busan_pump_data.csv', encoding='euc-kr')
pump_data2 = gpd.read_file('../Data/rawdata/FL_PUMP/FL_PUMP_SP.shp')
pump_data1.iloc[29:31,0:1] = '금정구'
pump_data1.iloc[26:27,0:1] = '연제구'
code['CODE'] = code['CODE'].apply(str)

In [72]:
code

,CODE,ZONE,DONG
0,2611000000,중구,NaN
1,2611010100,중구,영주동
2,2611010200,중구,대창동1가
3,2611010300,중구,대창동2가
4,2611010400,중구,중앙동1가
...,...,...,...
203,2671025000,기장군,기장읍
204,2671025300,기장군,장안읍
205,2671025600,기장군,정관읍
206,2671031000,기장군,일광면


In [73]:
pump_data2 = pd.merge(code, pump_data2, left_on='CODE', right_on='ADM_CD', how='inner')

In [74]:
pump_data2.shape

(21, 18)

In [75]:
pump_data2 = pump_data2.groupby(by='ZONE').count()['CODE']

In [76]:
pump_data2.sum()

21

In [77]:
pump_data1 = pump_data1.groupby(by='구군').count()['배수펌프장명']
pump_data = pump_data1.add(pump_data2,fill_value=0)

In [81]:
pump_data = pump_data.reset_index()
pump_data.rename(columns={'index':'ZONE',0:'PUMP_COUNT'},inplace=True)
pump_data['PUMP_COUNT'] = pump_data['PUMP_COUNT'].astype(int)

In [84]:
pump_data

,ZONE,PUMP_COUNT
0,강서구,22
1,금정구,2
2,기장군,3
3,남구,7
4,동구,2
5,동래구,4
6,북구,6
7,사상구,14
8,사하구,7
9,연제구,5


In [85]:
PIH_merge = pd.merge(PIH_merge,pump_data,on='ZONE',how='left')

In [86]:
PIH_merge = PIH_merge.fillna(0)

In [87]:
PIH_merge

,CODE,ZONE,IMP_SUR_RATIO,ZONE_AREA,MANHOLES_COUNT,PUMP_COUNT
0,2611000000,중구,81.61,3.010000e+07,773,0.0
1,2614000000,서구,40.57,1.414000e+08,1308,0.0
2,2617000000,동구,57.92,1.009000e+08,2410,2.0
3,2620000000,영도구,45.77,1.463000e+08,1914,0.0
4,2623000000,부산진구,45.91,2.978000e+08,6581,0.0
5,2626000000,동래구,59.03,1.667000e+08,3165,4.0
6,2629000000,남구,51.82,2.728000e+08,3156,7.0
7,2632000000,북구,22.17,3.976000e+08,6842,6.0
8,2635000000,해운대구,27.63,5.232000e+08,8794,11.0
9,2638000000,사하구,38.20,4.711000e+08,8331,7.0


In [89]:
PIH_merge['PUMP_RATIO'] = PIH_merge['PUMP_COUNT'] / PIH_merge['ZONE_AREA']

In [94]:
PIH_merge['MANHOLES_RATIO'] = PIH_merge['MANHOLES_COUNT'] * 1000 / PIH_merge['ZONE_AREA']

In [96]:
PIH_merge = PIH_merge.sort_values('ZONE').reset_index(drop=True)

In [98]:
PIH_merge.to_csv('../Data/processing_data/2nd/PIH_merge.csv', encoding='utf-8', index=False)